# Dashboard SQL Queries для Airbnb Analytics

Цей notebook містить SQL запити для створення дешборду на Databricks.
Всі запити використовують дані з Gold рівня.


In [0]:
%sql
-- Налаштування
USE CATALOG airbnb_lab3;
USE DATABASE airbnb_gold;


## 1. KPI Cards - Загальні метрики по містах (останній місяць)


In [0]:
%sql
-- Загальні KPI для останнього доступного місяця
SELECT 
    city,
    total_listings,
    ROUND(avg_nightly_price, 2) AS avg_price,
    ROUND(avg_occupancy_rate * 100, 2) AS occupancy_rate_pct,
    ROUND(avg_review_score, 2) AS avg_rating
FROM gold_city_month_metrics
WHERE yyyymm = (SELECT MAX(yyyymm) FROM gold_city_month_metrics)
ORDER BY city;


## 2. Тренди за часом - Місячні метрики


In [0]:
%sql
-- Тренд кількості оголошень за часом
SELECT 
    yyyymm,
    city,
    total_listings,
    LAG(total_listings) OVER (PARTITION BY city ORDER BY yyyymm) AS prev_month_listings,
    total_listings - LAG(total_listings) OVER (PARTITION BY city ORDER BY yyyymm) AS listings_change
FROM gold_city_month_metrics
ORDER BY yyyymm DESC, city;


In [0]:
%sql
-- Тренд середньої ціни за ніч та зайнятості
SELECT 
    yyyymm,
    city,
    ROUND(avg_nightly_price, 2) AS avg_price,
    ROUND(avg_occupancy_rate * 100, 2) AS occupancy_pct,
    ROUND(avg_review_score, 2) AS avg_rating
FROM gold_city_month_metrics
ORDER BY yyyymm DESC, city;


## 3. Порівняння міст - Cross-City Analysis


In [0]:
-- Детальне порівняння LA vs NYC
SELECT 
    yyyymm,
    LA_total_listings,
    NYC_total_listings,
    delta_listings,
    ROUND(LA_avg_price, 2) AS LA_price,
    ROUND(NYC_avg_price, 2) AS NYC_price,
    ROUND(delta_avg_price, 2) AS price_delta,
    ROUND(LA_occ_rate * 100, 2) AS LA_occupancy_pct,
    ROUND(NYC_occ_rate * 100, 2) AS NYC_occupancy_pct,
    ROUND(delta_occ_rate * 100, 2) AS occupancy_delta_pct
FROM gold_cross_city_compare
ORDER BY yyyymm DESC;


In [0]:
-- Відносне порівняння (LA як % від NYC)
SELECT 
    yyyymm,
    ROUND((LA_total_listings / NULLIF(NYC_total_listings, 0)) * 100, 1) AS listings_ratio_pct,
    ROUND((LA_avg_price / NULLIF(NYC_avg_price, 0)) * 100, 1) AS price_ratio_pct,
    ROUND((LA_occ_rate / NULLIF(NYC_occ_rate, 0)) * 100, 1) AS occupancy_ratio_pct
FROM gold_cross_city_compare
ORDER BY yyyymm DESC;


## 4. Топ райони за різними метриками


In [0]:
-- Топ 10 районів за кількістю оголошень (останній місяць)
SELECT 
    city,
    neighbourhood,
    listing_count,
    ROUND(avg_price, 2) AS avg_price,
    ROUND(avg_occupancy_rate * 100, 2) AS occupancy_pct
FROM gold_neighbourhood_metrics
WHERE yyyymm = (SELECT MAX(yyyymm) FROM gold_neighbourhood_metrics)
ORDER BY listing_count DESC
LIMIT 10;


In [0]:
-- Топ 10 найдорожчих районів
SELECT 
    city,
    neighbourhood,
    ROUND(avg_price, 2) AS avg_price,
    listing_count,
    ROUND(avg_occupancy_rate * 100, 2) AS occupancy_pct
FROM gold_neighbourhood_metrics
WHERE yyyymm = (SELECT MAX(yyyymm) FROM gold_neighbourhood_metrics)
    AND avg_price IS NOT NULL
ORDER BY avg_price DESC
LIMIT 10;


In [0]:
-- Топ 10 районів за зайнятістю
SELECT 
    city,
    neighbourhood,
    ROUND(avg_occupancy_rate * 100, 2) AS occupancy_pct,
    ROUND(avg_price, 2) AS avg_price,
    listing_count
FROM gold_neighbourhood_metrics
WHERE yyyymm = (SELECT MAX(yyyymm) FROM gold_neighbourhood_metrics)
    AND avg_occupancy_rate IS NOT NULL
ORDER BY avg_occupancy_rate DESC
LIMIT 10;


## 5. Розподіл типів кімнат


In [0]:
-- Поточний розподіл типів кімнат по містах
SELECT 
    city,
    room_type,
    cnt AS listing_count,
    ROUND(share * 100, 2) AS share_pct
FROM gold_roomtype_share
WHERE yyyymm = (SELECT MAX(yyyymm) FROM gold_roomtype_share)
ORDER BY city, share DESC;


In [0]:
-- Тренд зміни частки типів кімнат за часом
SELECT 
    yyyymm,
    city,
    room_type,
    ROUND(share * 100, 2) AS share_pct,
    cnt AS listing_count
FROM gold_roomtype_share
ORDER BY yyyymm DESC, city, share DESC;


## 6. Активність відгуків


In [0]:
-- Місячна активність відгуків
SELECT 
    yyyymm,
    city,
    review_count,
    LAG(review_count) OVER (PARTITION BY city ORDER BY yyyymm) AS prev_month_reviews,
    review_count - LAG(review_count) OVER (PARTITION BY city ORDER BY yyyymm) AS reviews_change
FROM gold_reviews_monthly
ORDER BY yyyymm DESC, city;


In [0]:
-- Загальна статистика відгуків по містах
SELECT 
    city,
    SUM(review_count) AS total_reviews,
    AVG(review_count) AS avg_monthly_reviews,
    MAX(review_count) AS max_monthly_reviews,
    MIN(review_count) AS min_monthly_reviews
FROM gold_reviews_monthly
GROUP BY city
ORDER BY total_reviews DESC;


## 7. Комплексний аналіз - Об'єднані метрики


In [0]:
-- Об'єднання метрик міста з відгуками для повної картини
SELECT 
    cm.yyyymm,
    cm.city,
    cm.total_listings,
    ROUND(cm.avg_nightly_price, 2) AS avg_price,
    ROUND(cm.avg_occupancy_rate * 100, 2) AS occupancy_pct,
    ROUND(cm.avg_review_score, 2) AS avg_rating,
    COALESCE(rm.review_count, 0) AS review_count,
    CASE 
        WHEN cm.total_listings > 0 THEN ROUND(COALESCE(rm.review_count, 0) / cm.total_listings, 2)
        ELSE 0 
    END AS reviews_per_listing
FROM gold_city_month_metrics cm
LEFT JOIN gold_reviews_monthly rm 
    ON cm.city = rm.city AND cm.yyyymm = rm.yyyymm
ORDER BY cm.yyyymm DESC, cm.city;


## 8. Аналіз сезонності


In [0]:
-- Середні метрики по місяцях (для виявлення сезонності)
SELECT 
    SUBSTRING(yyyymm, 6, 2) AS month_num,
    CASE SUBSTRING(yyyymm, 6, 2)
        WHEN '01' THEN 'Січень'
        WHEN '02' THEN 'Лютий'
        WHEN '03' THEN 'Березень'
        WHEN '04' THEN 'Квітень'
        WHEN '05' THEN 'Травень'
        WHEN '06' THEN 'Червень'
        WHEN '07' THEN 'Липень'
        WHEN '08' THEN 'Серпень'
        WHEN '09' THEN 'Вересень'
        WHEN '10' THEN 'Жовтень'
        WHEN '11' THEN 'Листопад'
        WHEN '12' THEN 'Грудень'
    END AS month_name,
    city,
    ROUND(AVG(avg_nightly_price), 2) AS avg_price_by_month,
    ROUND(AVG(avg_occupancy_rate) * 100, 2) AS avg_occupancy_by_month
FROM gold_city_month_metrics
GROUP BY SUBSTRING(yyyymm, 6, 2), city
ORDER BY month_num, city;


## 9. Швидкі статистики для швидкого огляду


In [0]:
-- Швидкий огляд: мінімум, максимум, середнє по всіх метриках
SELECT 
    city,
    MIN(total_listings) AS min_listings,
    MAX(total_listings) AS max_listings,
    ROUND(AVG(total_listings), 0) AS avg_listings,
    ROUND(MIN(avg_nightly_price), 2) AS min_price,
    ROUND(MAX(avg_nightly_price), 2) AS max_price,
    ROUND(AVG(avg_nightly_price), 2) AS avg_price,
    ROUND(MIN(avg_occupancy_rate) * 100, 2) AS min_occupancy_pct,
    ROUND(MAX(avg_occupancy_rate) * 100, 2) AS max_occupancy_pct,
    ROUND(AVG(avg_occupancy_rate) * 100, 2) AS avg_occupancy_pct
FROM gold_city_month_metrics
GROUP BY city;


## 10. Динаміка змін - MoM (Month-over-Month) зростання


In [0]:
-- MoM зростання/падіння ключових метрик
WITH monthly_data AS (
    SELECT 
        yyyymm,
        city,
        total_listings,
        avg_nightly_price,
        avg_occupancy_rate,
        LAG(total_listings) OVER (PARTITION BY city ORDER BY yyyymm) AS prev_listings,
        LAG(avg_nightly_price) OVER (PARTITION BY city ORDER BY yyyymm) AS prev_price,
        LAG(avg_occupancy_rate) OVER (PARTITION BY city ORDER BY yyyymm) AS prev_occupancy
    FROM gold_city_month_metrics
)
SELECT 
    yyyymm,
    city,
    ROUND(((total_listings - prev_listings) / NULLIF(prev_listings, 0)) * 100, 2) AS listings_growth_pct,
    ROUND(((avg_nightly_price - prev_price) / NULLIF(prev_price, 0)) * 100, 2) AS price_growth_pct,
    ROUND(((avg_occupancy_rate - prev_occupancy) / NULLIF(prev_occupancy, 0)) * 100, 2) AS occupancy_growth_pct
FROM monthly_data
WHERE prev_listings IS NOT NULL
ORDER BY yyyymm DESC, city;
